**Import des librairies**

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt

**Parametres CUDA pour modélisation en local**

In [ ]:
# Installer CUDA, CUDNN
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("Nombre de GPU disponible : ", len(gpus))

**Parametres divers**

In [ ]:
BATCH_SIZE = 10
N_EPOCHS = 150

ADAM_LEARNING_RATE = 0.0003
PATIENCE_ES = 12

IMG_HEIGHT = 224
IMG_WIDTH = 224

VALIDATION_RATIO = 0.2
TEST_RATIO = 0.1
TRAIN_RATIO = 1 - VALIDATION_RATIO - TEST_RATIO

N_BREEDS = 20
N_IMAGE_PER_CLASS = 140

RANDOM_STATE = 42

In [ ]:
random.seed(RANDOM_STATE)

**Préparation des dossiers pour la génération d'images et l'augmentation**

In [ ]:
# On se déplace dans le dossier images
os.chdir('data/images')

In [ ]:
# On supprimer les dossiers de modélisations et leurs contenus si déjà éxistants dans images
_ = [shutil.rmtree(path) for path in ["train","valid","test"] if os.path.isdir(path) is True ]

In [ ]:
# On recuperer les races (subdirectories)
list_dir_breeds = os.listdir()

In [ ]:
# On observe le nombre d'images pour chaque races dans leur dossier respectifs à l'aide d'un dataframe
df_breds = pd.DataFrame([[f"{path:40}",len(os.listdir(path))] for path in list_dir_breeds] , columns=["race", "nombre_images"])
df_breds

In [ ]:
# On selectionne au hasard un nombre N de races.
list_dir_breeds = random.sample(list_dir_breeds, N_BREEDS)
list_dir_breeds

Pour éviter d'augmenter les données lors de la séparation du jeu de validation de Keras avec ImageDataGenerator, il y a plusieurs techniques, je vais séparer le jeu puis créer différent ImageDataGenerator

In [ ]:
# Organiser les données en un dossier d'entrainement, de validation et de test
for dir_breeds in list_dir_breeds:
    path_train = f"train/{dir_breeds}"
    path_valid = f"valid/{dir_breeds}"
    path_test = f"test/{dir_breeds}"
    
    # On crée nos dossiers vides
    [os.makedirs(path) for path in [path_train,path_valid,path_test] if os.path.isdir(path) is False]

In [ ]:
# Si le nombre d'images minimum par classe est inférieur à notre paramétre on renvoit une erreur
min_images = df_breds["nombre_images"].min()
assert(min_images > N_IMAGE_PER_CLASS)

In [ ]:
train_size, valid_size, test_size = [int(TRAIN_RATIO*N_IMAGE_PER_CLASS) , int(VALIDATION_RATIO*N_IMAGE_PER_CLASS), int(TEST_RATIO*N_IMAGE_PER_CLASS)]

In [ ]:
# Train Test Split
for dir_breeds in list_dir_breeds:
    path_train = f"train/{dir_breeds}"
    path_valid = f"valid/{dir_breeds}"
    path_test = f"test/{dir_breeds}"
    
    # Si nos dossiers sont vides
    if len(os.listdir(path_train)+os.listdir(path_valid)+os.listdir(path_test)) == 0:

        list_path_images = os.listdir(path=dir_breeds)
        
        # On ajoute le nombre d'images choisi pour l'entrainement, la validation et le test
        for path_image in random.sample(list_path_images, train_size):
            shutil.copy(f"{dir_breeds}/{path_image}", path_train)
            list_path_images.remove(path_image)

        for path_image in random.sample(list_path_images, valid_size):
            shutil.copy(f"{dir_breeds}/{path_image}", path_valid)
            list_path_images.remove(path_image)

        for path_image in random.sample(list_path_images, test_size):
            shutil.copy(f"{dir_breeds}/{path_image}", path_test)
            list_path_images.remove(path_image)
    else:
        print("Les dossiers ne sont pas vides")
        break

In [ ]:
# On revient dans le dossier root
os.chdir('../../')

In [ ]:
# Chemins de nos dossiers fraichement générer
train_path = "data/images/train"
valid_path = "data/images/valid"
test_path = "data/images/test"

In [ ]:
# Création des itérateurs de données pour notre modélisation
print("train batches :")
train_image_data_generator = ImageDataGenerator(rescale=1./255,
                                                rotation_range=40,
                                                width_shift_range=0.2,
                                                height_shift_range=0.2,
                                                shear_range=0.2,
                                                zoom_range=0.2,
                                                horizontal_flip=True)
train_batches = train_image_data_generator.flow_from_directory(directory=train_path,
                                                               target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                               classes=list_dir_breeds,
                                                               batch_size=BATCH_SIZE)

print("valid batches :")
valid_image_data_generator = ImageDataGenerator(rescale=1./255)
valid_batches = valid_image_data_generator.flow_from_directory(directory=valid_path,
                                                               target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                               classes=list_dir_breeds,
                                                               batch_size=BATCH_SIZE)


print("test batches :")
test_image_data_generator = ImageDataGenerator(rescale=1./255)
test_batches = test_image_data_generator.flow_from_directory(directory=test_path,
                                                             target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                             classes=list_dir_breeds,
                                                             batch_size=BATCH_SIZE,
                                                             shuffle=False)

**Analyse du nombre d'images par race**

In [ ]:
import plotly.express as px

fig = px.bar(df_breds, orientation='h', y='race', x="nombre_images", width=800, height=800)
fig.update_layout(showlegend=False)
fig.show()

**Visualisation de l'augmentation avant entrainement**

In [ ]:
# On génére le prochain batch du train avec augmentation
imgs, labels = next(train_batches)

In [ ]:
# Fonction custom pour visualiser l'augmentation
def plot_images(images_arr, labels=None, rescaled=True,print_shape=True):
    if rescaled:
        images_arr= images_arr*255.0
    size = len(images_arr)
    fig, axes = plt.subplots(1, size, figsize=(20,20))
    axes = axes.flatten()
    for i, (img, ax) in enumerate(zip( images_arr, axes)):
        img = img.astype(np.uint8)
        ax.imshow(img)
        if labels is not None:
            ax.set_title(labels[i])
        elif print_shape is True :
            ax.set_title(np.array(img).shape)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_images(imgs, print_shape=True)

**Modélisation avec CNN**

In [ ]:
model = Sequential([
    Conv2D(filters = 32,
           kernel_size = (3, 3),
           activation = 'relu',
           padding = 'same',
           input_shape = (IMG_HEIGHT,IMG_WIDTH,3)
          ),
    MaxPool2D(pool_size = (2, 2),
              strides=2
             ),
    Conv2D(filters = 64,
           kernel_size=(3, 3),
           activation = 'relu',
           padding = 'same'
          ),
    MaxPool2D(pool_size = (2, 2),
              strides = 2
             ),
    Flatten(),
    Dense(units = 160,
          activation = 'relu'
         ),
    Dense(units = N_BREEDS,
          activation = 'softmax'
         )
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate=ADAM_LEARNING_RATE),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
             )

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
                                     patience = PATIENCE_ES
                                    )
]

In [ ]:
history = model.fit(x = train_batches,
                    steps_per_epoch = len(train_batches),
                    validation_data = valid_batches,
                    validation_steps = len(valid_batches),
                    epochs = N_EPOCHS,
                    verbose = 2,
                    callbacks = my_callbacks
                   )

In [ ]:
#Afficher l'évolution par epoch
def plot_evolution(history):
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.title(f"Evolution de l'accuracy pour {N_BREEDS} races")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "val"], loc="upper left")
    plt.show()

In [ ]:
plot_evolution(history)

**Transfert Learning**

[VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION arxiv.org/pdf](https://arxiv.org/pdf/1409.1556.pdf)

During training, the input to our ConvNets is a fixed-size 224 × 224 RGB image.
The only preprocessing we do is subtracting the mean RGB value, computed on the training set, from each pixel.
The image is passed through a stack of convolutional (conv.) layers, where we use filters with a very
small receptive field: 3 × 3 (which is the smallest size to capture the notion of left/right, up/down,
center).
In one of the configurations we also utilise 1 × 1 convolution filters, which can be seen as
a linear transformation of the input channels (followed by non-linearity). The convolution stride is
fixed to 1 pixel; the spatial padding of conv. layer input is such that the spatial resolution is preserved
after convolution, i.e. the padding is 1 pixel for 3 × 3 conv. layers. Spatial pooling is carried out by
five max-pooling layers, which follow some of the conv. layers (not all the conv. layers are followed
by max-pooling). Max-pooling is performed over a 2 × 2 pixel window, with stride 2.
A stack of convolutional layers (which has a different depth in different architectures) is followed by
three Fully-Connected (FC) layers: the first two have 4096 channels each, the third performs 1000-
way ILSVRC classification and thus contains 1000 channels (one for each class). The final layer is
the soft-max layer. The configuration of the fully connected layers is the same in all networks.
All hidden layers are equipped with the rectification (ReLU (Krizhevsky et al., 2012)) non-linearity.
We note that none of our networks (except for one) contain Local Response Normalisation
(LRN) normalisation (Krizhevsky et al., 2012): as will be shown in Sect. 4, such normalisation
does not improve the performance on the ILSVRC dataset, but leads to increased memory consumption and computation time. Where applicable, the parameters for the LRN layer are those
of (Krizhevsky et al., 2012).

In [ ]:
# On récupere le modele avec les poids entrainés
vgg16 = VGG16(weights = "imagenet",
              include_top = True
             )

In [ ]:
# On crée notre derniere couche à partir des couches précédentes de vgg16
last_layer = Dense(units = N_BREEDS,
                   activation ='softmax',
                   name ='predictions'
                  )(vgg16.layers[-2].output)

# Puis on crée notre modéle
model = tf.keras.Model(inputs = vgg16.input,
                       outputs = last_layer
                      )

In [ ]:
for layer in model.layers:
    layer.trainable = False

model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.0003),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
             )

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = "val_loss",
                                     patience = 12
                                    )
]

In [ ]:
history = model.fit(x = train_batches,
                    steps_per_epoch = len(train_batches),
                    validation_data = valid_batches,
                    validation_steps = len(valid_batches),
                    epochs = N_EPOCHS,
                    verbose = 2,
                    callbacks = my_callbacks
                   )

In [ ]:
plot_evolution(history)